# 处理分类数据

In [9]:
import numpy as np
import pandas as pd 

In [10]:
# 创建一个数据集,包含无序特征(color),有序特征(size),数值型特征(prize),最后一列储存的是类别
df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class3']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class3


### 映射有序特征

In [11]:
# 将有序特征转化成数字形式,保证学习算法能够正确解释
size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1
}

# 利用map方法将对应关系映射到size中,生成新的一列避免原数据被破坏
df['size_map'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel,size_map
0,green,M,10.1,class1,1
1,red,L,13.5,class2,2
2,blue,XL,15.3,class3,3


In [12]:
# 如果直接替换size列可以利用反字典来还原原来的数据
inv_size_mapping = {
    v: k for k, v in size_mapping.items()
}

inv_size_mapping

{1: 'M', 2: 'L', 3: 'XL'}

### 对类别进行编码

In [17]:
# 手动创建映射字典
class_mapping = {
    label:idx for idx, label in enumerate(np.unique(df['classlabel']))
}
class_mapping

{'class1': 0, 'class2': 1, 'class3': 2}

In [19]:
df['classlabel_map'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel,size_map,classlabel_map
0,green,M,10.1,class1,1,0
1,red,L,13.5,class2,2,1
2,blue,XL,15.3,class3,3,2


In [20]:
# sklearn中也提供了LabelEncoder类来实现类别的转换
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()

In [24]:
# fit_transform方法是fit和transform方法的合并
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 2])

In [25]:
# 我们还可以调用inverse_transform方法得到原始的字符串类型
class_le.inverse_transform(y)

/Users/pengyi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['class1', 'class2', 'class3'], dtype=object)

### 对离散特征进行独热编码

In [37]:
# 使用LabelEncoder来转换无序的离散特征
x = df[['color', 'size_map', 'price']].values
x

array([['green', 1, 10.1],
       ['red', 2, 13.5],
       ['blue', 3, 15.3]], dtype=object)

In [38]:
color_le = LabelEncoder()

In [39]:
# 将无序离散特征"color"转换成整形
x[:, 0] = color_le.fit_transform(x[:, 0])
x

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

虽然已经将"color"转换成整型了,但是我们犯了一个错误

原来的数据是不含顺序的,但是转换成整型之后,__学习算法会认为‘green’比‘blue’大，‘red’比‘green’大__,这显然是不对的,因为__颜色是无序的__

所以处理无序特征的时候,需要用__独热编码__来处理

__独热编码__:独热编码会为每个离散值创建一个哑特征(dummy feature)。什么是哑特征呢？举例来说，对于‘颜色’这一特征中的‘蓝色’，我们将其编码为[蓝色=1，绿色=0，红色=0]，同理，对于‘绿色’，我们将其编码为[蓝色=0，绿色=1，红色=0]，特点就是向量只有一个1，其余均为0，故称之为one-hot。

In [40]:
# 使用OneHotEncoder来实现独热编码
from sklearn.preprocessing import OneHotEncoder

In [41]:
# 初始化OneHotEncoder并通过categorical_features参数设置要进行独热编码的列
ohe = OneHotEncoder(categorical_features=[0])

In [43]:
# fit_transform默认返回稀疏矩阵,所以调用toarray()方法来将稀疏矩阵转化成一般矩阵
ohe.fit_transform(x).toarray()

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [44]:
ohe.fit_transform(x)

<3x5 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in COOrdinate format>

除了使用sklearn中的OneHotEncoder类得到哑特征,还可以使用pandas中的get_dummies方法来创建哑特征(__推荐__)